# Introduction

Ce notebook permet le nettoyage des données, et retourne le Dataframe nettoyé qui sera utilisé pour l'analyse

## Importation

In [ ]:
!pip install deep-translator
!pip install spacy
!pip install translate
!pip install googletrans

In [ ]:
import spacy
import pandas as pd
from translate import Translator
import nltk
nltk.download('stopwords')
import string
from re import sub
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from deep_translator import GoogleTranslator
from nltk.stem.snowball import SnowballStemmer
!python -m spacy download fr_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-20 06:54:52.426749: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45.8 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [ ]:
translator = GoogleTranslator(source='auto', target='fr')
nlp=spacy.load('fr_core_news_md')

## convert_emojis_to_word:

permet de transformer les emojis d'un texte en mots, on préfère les garder pour mieux analyser les sentiments

In [ ]:


try:
    import cPickle as pickle
except ImportError: 
    import pickle
import re

with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        if emot in text:
            convert=" ".join(Emoji_Dict[emot].replace(",","").replace(":","").replace("_"," ").split())
            text = re.sub(r'('+emot+')', translator.translate(convert) , text)
    return text


## les stopwords:

c'est une liste de mots qui n'exprime pas de sentiment dans notre contexte . Une partie est déjà défini dans nltk, on va juste rajouter ceux qui nous semble inutiles par rapport au contexte.

In [ ]:
STOPWORDS = stopwords.words('french')
STOPWORDS += list(string.punctuation)+list(string.ascii_uppercase)+list(string.ascii_lowercase)+['vacciné','vaccin','russe','ukrainien','peuple',
           'Macron','Lola','rouler','president','le','la','energie','chomage',
           'election','retraite','ecologique','ecologie','climat',
           'faire','vous','aller','cela','France','plus','visage',
           'président','françai','comme','wallet','quand',
           'quoi','dire','vouloir','falloir','pouvoir',
           'devoir','venir','joievisage','être', 
           'Afrique', 'Poutine','alors','Europe','vacciner',
           'Monsieur','pourquoi','français','savoir','avoir',
           'pays','ukraine','poutine','russie','guerre','Arménie']
STOPWORDS = set(STOPWORDS)

In [ ]:
def lemme(tweet):
  doc = nlp(tweet)
  empty_list = []
  for token in doc:
    empty_list.append(token.lemma_)
  final_string = ' '.join(map(str,empty_list))
  return final_string

##  text_to_word_list:

permet de nettoyer les tweets  de la manière suivante:

+ suppression des doublons
+ transformation des emojis en texte
+ supprimer les liens
+ supprimer des retour à la ligne
+ suppresion des tags
+ suppression des htags
+ supression des apostrophes
+ suppression des mails
+ suppression des chaînes de caractères contenant des nombres
+ suppression des stopwords
+ lemmatisation et steming qui consiste entre autres à transformer les abréviations conventionnelles en leur réelles expressions, à mettre les verbes à l'infinitif pour éviter la répétition...

In [ ]:

def text_to_word_list(tweets):
     
    tweets= tweets.apply(lambda tweet:convert_emojis_to_word(tweet))
    tweets=tweets.replace(re.compile(r"((www\.[^\s]+)|(http(s)?://[^\s]+))"), "")
    tweets = tweets.str.replace("\n", " ").replace("\r", " ") 
    tweets=tweets.str.replace("@[\w]*","")
  
    tweets=tweets.str.replace(re.compile(r"[,\!\?\%\(\)\/\"\$\£\+\*\µ,\:\&\§\.]"), " ")
    tweets=tweets.str.replace(re.compile(r"([#]\w+)"),"")
    tweets=tweets.str.replace(re.compile(r"([\’])")," ")
    tweets=tweets.str.replace(re.compile(r"([\'])")," ")
    tweets=tweets.str.replace(re.compile(r"\S+@\S+"),"")
    tweets=tweets.str.replace(re.compile(r"([.]{1,5})"),"")
    tweets= tweets.str.replace(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*","")
    

    tweets= tweets.str.split()
    tweets= tweets.apply(lambda tweet: [word for word in tweet if word not in STOPWORDS])
    tweets= tweets.apply(lambda tweet: " ".join(tweet))
    tweets= tweets.apply(lambda tweet: lemme(tweet))
    return tweets 

# cleaned_all_data: 

exécute le nettoyage du dataframe et le sauvegarde dans df_clean 

In [ ]:

def cleaned_all_data(df):
  df.drop_duplicates(subset=['Tweet'])
  df['sl_clean']=text_to_word_list(df['Tweet'])
  df.to_csv("df_clean.csv",index=False)
  return df

# read_all_data:

recupère les données scrappées

In [ ]:
def read_all_data():

  df=pd.read_csv("../scrapping/df.csv")
  return df

In [ ]:
df=cleaned_all_data(read_all_data())

<ipython-input-7-84a4862e8efe>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets=tweets.str.replace("@[\w]*","")
<ipython-input-7-84a4862e8efe>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets= tweets.str.replace(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*","")
